# Decision Layer Walkthrough

The decision layer library can be used as a super fast decision making layer on top of LLMs. That means that rather than waiting on a slow agent to decide what to do, we can use the magic of semantic vector space to make decisions. Cutting decision making time down from seconds to milliseconds.

## Getting Started

In [14]:
!pip install -qU \
    decision-layer


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


We start by defining a dictionary mapping decisions to example phrases that should trigger those decisions.

In [15]:
from decision_layer.schema import Decision

politics = Decision(
    name="politics",
    utterances=[
        "isn't politics the best thing ever",
        "why don't you tell me about your political opinions",
        "don't you just love the president"
        "don't you just hate the president",
        "they're going to destroy this country!",
        "they will save the country!",
        "did you hear about the new goverment proposal regarding the ownership of cats and dogs",
    ]
)

Let's define another for good measure:

In [16]:
chitchat = Decision(
    name="chitchat",
    utterances=[
        "how's the weather today?",
        "how are things going?",
        "lovely weather today",
        "the weather is horrendous",
        "let's go to the chippy",
        "it's raining cats and dogs",
    ]
)

decisions = [politics, chitchat]

Now we initialize our embedding model (we will add support for Hugging Face):

In [17]:
from decision_layer.encoders import OpenAIEncoder
import os

encoder = OpenAIEncoder(name="text-embedding-ada-002")

Now we define the `DecisionLayer`. When called, the decision layer will consume text (a query) and output the category (`Decision`) it belongs to — for now we can only `_query` and get the most similar `Decision` `utterances`.

In [18]:
from decision_layer import DecisionLayer

dl = DecisionLayer(encoder=encoder, decisions=decisions)

In [26]:
out = dl._query("don't you love politics?")
out

[{'decision': 'politics', 'score': 0.22792677421560453},
 {'decision': 'politics', 'score': 0.2315237823644528},
 {'decision': 'politics', 'score': 0.2516642096551168},
 {'decision': 'politics', 'score': 0.2531645714220874},
 {'decision': 'politics', 'score': 0.2566108224655662}]

Using the most similar `Decision` `utterances` and their `cosine similarity scores`, use `simple_classify` to apply scoring a secondary scoring system which chooses the `decision` that the utterance belongs to.

Here we use `apply_tan=True`, which means that a `tan` function is assigned to each score boosting the score of `decisions` whose datapoints had greater `cosine similarlity` and reducing the score of those which had lower `cosine similarity`. 

In [27]:
decision, scores_by_category = dl.simple_classify(query_results=out, apply_tan=True)
decision

'politics'

In [28]:
scores_by_category

{'politics': 2.018519173992354}

The correct category was chosen. Let's try again for a less clear-cut case:

In [31]:
out = dl._query("i love cats and dogs!")
out

[{'decision': 'chitchat', 'score': 0.22320888353212376},
 {'decision': 'politics', 'score': 0.22367029584935166},
 {'decision': 'politics', 'score': 0.2274250403127478},
 {'decision': 'politics', 'score': 0.23451692377042876},
 {'decision': 'chitchat', 'score': 0.24924083653953585}]

In [32]:
decision, scores_by_category = dl.simple_classify(query_results=out, apply_tan=True)
decision

'politics'

In [33]:
scores_by_category

{'chitchat': 0.7785435459589187, 'politics': 1.1258003022715952}

In [34]:
dl.categories

array(['politics', 'politics', 'politics', 'politics', 'politics',
       'politics', 'chitchat', 'chitchat', 'chitchat', 'chitchat',
       'chitchat', 'chitchat'], dtype='<U8')